In [ ]:
response = llm.invoke("What is tool calling in langchain?")
print("\nResponse Content: ", response.content)

In [ ]:
%%capture
# Installing the core libraries I need for this math assistant project.
# I’m keeping it simple and fully open-source so it runs fine in Colab.

!pip install langchain langchain-community transformers wikipedia


In [ ]:
# For this project I don’t want to rely on IBM or OpenAI APIs,
# so I’m loading a small open-source model from Hugging Face instead.
# This keeps everything reproducible and easy to run in Colab.

model_id = "microsoft/phi-2"  # small enough to be Colab-friendly, good at reasoning-style tasks

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",   # lets Colab decide CPU/GPU placement
    torch_dtype="auto"   # keeps memory usage manageable
)

# Wrapping the model into a LangChain-compatible LLM interface
generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
)

llm = HuggingFacePipeline(pipeline=generation_pipeline)


In [ ]:
# Quick check to make sure the model is wired correctly before I start adding tools and agents.

response = llm("What is tool calling in LangChain?")
print("\nResponse Content:\n")
print(response)


In [ ]:
# This is the helper function my agent will eventually call.
# It takes a string, pulls out any numbers it finds, and returns their sum.

def add_numbers(inputs: str) -> dict:
    """
    Extracts numbers from a string and returns their sum in a dictionary.

    Example:
    "Add 10 and 20" → {"result": 30}
    """

    # I'm doing a simple parse here: remove commas, split the string,
    # keep only pieces that are digits, and convert them to integers.
    numbers = [int(x) for x in inputs.replace(",", "").split() if x.isdigit()]

    # Summing everything I extracted
    result = sum(numbers)

    return {"result": result}


In [ ]:
# Quick sanity check to confirm the function works before I wrap it as a tool.
add_numbers("1 2")


In [ ]:
# Wrapping my add_numbers function into a LangChain Tool.
# This is one way of exposing Python functions for agent use.

from langchain.agents import Tool

add_tool = Tool(
    name="AddTool",
    func=add_numbers,
    description="Extracts numbers from text and returns their sum."
)

print("Tool object:", add_tool)


In [ ]:
# Just taking a look at what this Tool object exposes.

print("Tool Name:")
print(add_tool.name)

print("\nTool Description:")
print(add_tool.description)

print("\nTool Function:")
print(add_tool.invoke)


In [ ]:
# Quick check to make sure this Tool wrapper actually works as expected.

print("Calling Tool Function:")
test_input = "10 20 30 a b"
print(add_tool.invoke(test_input))


In [ ]:
# The @tool decorator is the second way to create tools.
# This approach also gives me an autogenerated schema.

from langchain_core.tools import tool
import re

@tool
def add_numbers(inputs: str) -> dict:
    """
    Extracts numbers from a string and returns their sum.
    Example:
    "Add 10, 20, and 30" → {"result": 60}
    """
    numbers = [int(num) for num in re.findall(r'\d+', inputs)]
    result = sum(numbers)
    return {"result": result}


In [ ]:
print("Name:\n", add_numbers.name)
print("\nDescription:\n", add_numbers.description)
print("\nArgs:\n", add_numbers.args)


In [ ]:
test_input = "what is the sum between 10, 20 and 30"
print(add_numbers.invoke(test_input))


In [ ]:
# Quick comparison to understand how the constructor vs decorator differ.

print("Tool Constructor Approach:")
print(f"Has Schema: {hasattr(add_tool, 'args_schema')}\n")

print("@tool Decorator Approach:")
print(f"Has Schema: {hasattr(add_numbers, 'args_schema')}")
print(f"Args Schema Info: {add_numbers.args}")


In [ ]:
from typing import List

@tool
def add_numbers_with_options(numbers: List[float], absolute: bool = False) -> float:
    """
    Adds a list of numbers. If 'absolute' is True, takes absolute values first.
    """
    if absolute:
        numbers = [abs(n) for n in numbers]
    return sum(numbers)


In [ ]:
print("Args Schema (options tool):", add_numbers_with_options.args)
print("Args Schema (basic tool):", add_numbers.args)


In [ ]:
print(add_numbers_with_options.invoke({"numbers": [-1.1, -2.1, -3.0], "absolute": False}))
print(add_numbers_with_options.invoke({"numbers": [-1.1, -2.1, -3.0], "absolute": True}))


In [ ]:
from typing import Dict, Union

# This tool is a bit more flexible: it extracts integers and decimals,
# and returns either a numeric sum or a message if nothing is found.

@tool
def sum_numbers_with_complex_output(inputs: str) -> Dict[str, Union[float, str]]:
    """
    Extracts and sums all integers and decimal numbers from the input string.
    Returns either the numeric sum or a message if no numbers are found.
    """
    matches = re.findall(r'-?\d+(?:\.\d+)?', inputs)
    if not matches:
        return {"result": "No numbers found in input."}
    try:
        numbers = [float(num) for num in matches]
        total = sum(numbers)
        return {"result": total}
    except Exception as e:
        return {"result": f"Error during summation: {str(e)}"}


In [ ]:
# This is a simpler variant: it just returns the raw sum as a float.

@tool
def sum_numbers_from_text(inputs: str) -> float:
    """
    Adds a list of numbers provided in the input string.

    Args:
        inputs: A string containing numbers that should be extracted and summed.
    Returns:
        The sum of all numbers found in the input.
    """
    numbers = [int(num) for num in re.findall(r'\d+', inputs)]
    result = sum(numbers)
    return result


In [ ]:
from langchain.agents import initialize_agent

# This agent uses the original AddTool and my open-source LLM.
# It follows the classic ReAct-style pattern: think → act → observe → answer.

agent = initialize_agent(
    tools=[add_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
)


In [ ]:
# Letting the agent read the text, decide it needs math, and call the tool.

response = agent.run(
    "In 2023, the US GDP was approximately $27.72 trillion, "
    "while Canada's was around $2.14 trillion and Mexico's was about $1.79 trillion. "
    "What is the total?"
)


In [ ]:
response


In [ ]:
# Testing how the agent handles slightly messy numeric input.

agent.invoke({"input": "Add 10, 20, two and 30"})


In [ ]:
# Now I’m creating a second agent that uses the sum_numbers_from_text tool.
# This one uses the structured-chat style, which is better aligned with tools.

agent_2 = initialize_agent(
    tools=[sum_numbers_from_text],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
)

response = agent_2.invoke({"input": "Add 10, 20 and 30"})
print(response)


In [ ]:
# Here I’m building an agent around the more advanced tool
# that can also handle decimals and negative numbers.

agent_3 = initialize_agent(
    tools=[sum_numbers_with_complex_output],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
)

response = agent_3.invoke({"input": "Add 10, 20.5 and -3"})
print(response)


In [ ]:
# Now I’m wiring up the tool that supports an 'absolute' flag,
# so the agent can decide whether to use absolute values or not.

agent_2 = initialize_agent(
    tools=[add_numbers_with_options],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

response = agent_2.invoke({
    "input": "Add -10, -20, and -30 using absolute values."
})
print(response)


In [ ]:
"""
If I want to run the same tools using OpenAI in the future,
I can uncomment and adapt the code below (requires OPENAI_API_KEY).

from langchain_openai import ChatOpenAI

llm_ai = ChatOpenAI(model="gpt-4.1-nano")

agent_openai = initialize_agent(
    tools=[add_numbers_with_options],
    llm=llm_ai,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

response = agent_openai.invoke({
    "input": "Add -10, -20, and -30 using absolute values."
})
print(response)
"""


In [ ]:
%%capture
# Installing LangGraph so I can use its prebuilt ReAct-style agents.
!pip install langgraph==0.6.6


In [ ]:
from langgraph.prebuilt import create_react_agent

# Here I’m using a prebuilt ReAct agent from LangGraph.
# It will use my open-source LLM plus the sum_numbers_from_text tool.

agent_exec = create_react_agent(
    model=llm,
    tools=[sum_numbers_from_text],
)

msgs = agent_exec.invoke({
    "messages": [("human", "Add the numbers -10, -20, -30")]
})



In [ ]:
# Checking what the agent responded with in the last message.

print(msgs["messages"][-1].content)


In [ ]:
# Now I’m adding a subtraction tool so the agent can handle more than just addition.

@tool
def subtract_numbers(inputs: str) -> dict:
    """
    Extracts numbers from a string, negates the first number,
    and then subtracts the remaining numbers in sequence.
    """
    numbers = [int(num) for num in inputs.replace(",", "").split() if num.isdigit()]

    if not numbers:
        return {"result": 0}

    # Start with the first number negated
    result = -1 * numbers[0]

    for num in numbers[1:]:
        result -= num

    return {"result": result}


In [ ]:
print("Name:\n", subtract_numbers.name)
print("\nDescription:\n", subtract_numbers.description)
print("\nArgs:\n", subtract_numbers.args)


In [ ]:
print("Calling subtract_numbers Tool:")
test_input = "10 20 30 and four a b"
print(subtract_numbers.invoke(test_input))


In [ ]:
# Multiplication tool so the agent can handle product operations as well.

@tool
def multiply_numbers(inputs: str) -> dict:
    """
    Extracts numbers from a string and calculates their product.
    """
    numbers = [int(num) for num in inputs.replace(",", "").split() if num.isdigit()]
    print("Parsed numbers:", numbers)

    if not numbers:
        return {"result": 1}

    result = 1
    for num in numbers:
        result *= num
        print("Multiplying by:", num)

    return {"result": result}


In [ ]:
# Division tool for sequential division.

@tool
def divide_numbers(inputs: str) -> dict:
    """
    Extracts numbers from a string and divides the first number
    by each subsequent number in sequence.
    """
    numbers = [int(num) for num in inputs.replace(",", "").split() if num.isdigit()]

    if not numbers:
        return {"result": 0}

    result = numbers[0]
    for num in numbers[1:]:
        result /= num

    return {"result": result}


In [ ]:
# Testing the multiplication tool.

multiply_test_input = "2, 3, and four"
multiply_result = multiply_numbers.invoke(multiply_test_input)

print("--- Testing MultiplyTool ---")
print(f"Input: {multiply_test_input}")
print(f"Output: {multiply_result}")


In [ ]:
# Testing the division tool.

divide_test_input = "100, 5, two"
divide_result = divide_numbers.invoke(divide_test_input)

print("--- Testing DivideTool ---")
print(f"Input: {divide_test_input}")
print(f"Output: {divide_result}")


In [ ]:
# Grouping all my math tools together so the agent has multiple options.

tools = [add_numbers, subtract_numbers, multiply_numbers, divide_numbers]
tools


In [ ]:
from langgraph.prebuilt import create_react_agent

# Creating a math-focused ReAct agent with all tools wired in.
math_agent = create_react_agent(
    model=llm,
    tools=tools,
    # This prompt nudges the agent to be clear and deliberate.
    prompt="You are a helpful mathematical assistant that can perform various operations. Use the tools precisely and explain your reasoning clearly."
)


In [ ]:
response = math_agent.invoke({
    "messages": [("human", "What is 25 divided by 4?")]
})

final_answer = response["messages"][-1].content
print(final_answer)


In [ ]:
response_2 = math_agent.invoke({
    "messages": [("human", "Subtract 100, 20, and 10.")]
})

final_answer_2 = response_2["messages"][-2].content
print(final_answer_2)


In [ ]:
print("\n--- Testing MultiplyTool ---")
response = math_agent.invoke({
    "messages": [("human", "Multiply 2, 3, and four.")]
})
print("Agent Response:", response["messages"][-1].content)

print("\n--- Testing DivideTool ---")
response = math_agent.invoke({
    "messages": [("human", 'Divide 100 by 5 and then by 2.')]
})
print("Agent Response:", response["messages"][-1].content)


In [ ]:
# This is a more intuitive subtraction tool: it does x1 - x2 - x3 instead of negating the first.

@tool
def new_subtract_numbers(inputs: str) -> dict:
    """
    Extracts numbers from a string and performs sequential subtraction:
    first - second - third - ...
    """
    numbers = [int(num) for num in inputs.replace(",", "").split() if num.isdigit()]

    if not numbers:
        return {"result": 0}

    result = numbers[0]
    for num in numbers[1:]:
        result -= num

    return {"result": result}


In [ ]:
tools_updated = [add_numbers, new_subtract_numbers, multiply_numbers, divide_numbers]

math_agent_new = create_react_agent(
    model=llm,
    tools=tools_updated,
    prompt="You are a helpful mathematical assistant that can perform various operations. Use the tools precisely and explain your reasoning clearly."
)

print("math_agent_new:", math_agent_new)


In [ ]:
# I’m adding a small test suite so I can see how reliably the agent uses each tool.

test_cases = [
    {
        "query": "Subtract 100, 20, and 10.",
        "expected": {"result": 70},
        "description": "Testing subtraction tool with sequential subtraction."
    },
    {
        "query": "Multiply 2, 3, and 4.",
        "expected": {"result": 24},
        "description": "Testing multiplication tool for a list of numbers."
    },
    {
        "query": "Divide 100 by 5 and then by 2.",
        "expected": {"result": 10.0},
        "description": "Testing division tool with sequential division."
    },
    {
        "query": "Subtract 50 from 20.",
        "expected": {"result": -30},
        "description": "Testing subtraction tool with negative results."
    }
]


In [ ]:
correct_tasks = []

# Running each test case and checking whether the agent used the right tool
# and produced the expected result.

for index, test in enumerate(test_cases, start=1):
    query = test["query"]
    expected_result = test["expected"]["result"]

    print(f"\n--- Test Case {index}: {test['description']} ---")
    print(f"Query: {query}")

    response = math_agent_new.invoke({"messages": [("human", query)]})

    tool_message = None
    for msg in response["messages"]:
        if hasattr(msg, "name") and msg.name in [
            "add_numbers",
            "new_subtract_numbers",
            "multiply_numbers",
            "divide_numbers",
        ]:
            tool_message = msg
            break

    if tool_message:
        import json
        tool_result = json.loads(tool_message.content)["result"]
        print(f"Tool Result: {tool_result}")
        print(f"Expected Result: {expected_result}")

        if tool_result == expected_result:
            print(f"✅ Test Passed: {test['description']}")
            correct_tasks.append(test["description"])
        else:
            print(f"❌ Test Failed: {test['description']}")
    else:
        print("❌ No tool was called by the agent")

print("\nCorrectly passed tests:", correct_tasks)


In [ ]:
from langchain_community.utilities import WikipediaAPIWrapper

# Adding one non-math tool so the agent can mix retrieval with calculations.

@tool
def search_wikipedia(query: str) -> str:
    """Searches Wikipedia and returns a short summary."""
    wiki = WikipediaAPIWrapper()
    return wiki.run(query)


In [ ]:
tools_updated = [
    add_numbers,
    new_subtract_numbers,
    multiply_numbers,
    divide_numbers,
    search_wikipedia,
]

math_agent_updated = create_react_agent(
    model=llm,
    tools=tools_updated,
    prompt=(
        "You are a helpful assistant that can perform various mathematical operations "
        "and look up information. Use the tools precisely and explain your reasoning clearly."
    ),
)


In [ ]:
query = "What is the population of Canada? Multiply it by 0.75"

response = math_agent_updated.invoke({"messages": [("human", query)]})

print("\nMessage sequence:")
for i, msg in enumerate(response["messages"]):
    print(f"\n--- Message {i+1} ---")
    print(f"Type: {type(msg).__name__}")
    if hasattr(msg, "content"):
        print(f"Content: {msg.content}")
    if hasattr(msg, "name"):
        print(f"Name: {msg.name}")
    if hasattr(msg, "tool_calls") and msg.tool_calls:
        print(f"Tool calls: {msg.tool_calls}")
